In [ ]:
import os
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np

%matplotlib

In [ ]:
dataFile = '/Users/dgrossman/data/spmf_fpgrowth_output.txt'
dataFile = '/Users/dgrossman/work/netStorage/davec/tbird.TFIDF.60s.FPGrowth.minsup002'

dataFile = '/Users/dgrossman/Downloads/PARIS_Results_500k.py'

#dataFile = '/Users/dgrossman/work/netStorage/davec/8Jan2016/tbird.log.preProc.200.supports.30sec.Transactions.FPGrowth.3minsup'
#dataFile = '/Users/dgrossman/work/netStorage/davec/8Jan2016/tbird.log.preProc.200.supports.20sec.Transactions.FPGrowth.02minsup'


dataClusters = '/Users/dgrossman/data/tbird.log.preProc.200.supports.clusters'

nodeFile = open(dataFile,'r')
clusterFile = open(dataClusters,'r')


In [ ]:
import re
edgeSet = set()
edgeDict = defaultdict(int)
procLine = list()
clusterDict = dict()

for line in clusterFile:
    cluster,text = line.strip().split(',',1)
    text = re.sub(r'\\','',text,re.MULTILINE| re.DOTALL)
    text2 = re.sub(r',','',text,re.MULTILINE | re.DOTALL)
    clusterDict[cluster] = text2
    
clusterDict['-1'] = 'JunkDrawer'
    
for line in nodeFile:
    if re.search('#SUP',line):
        procLine.append(line.strip().split('#',1)[0].strip())
    else:
        procLine.append(line.strip())

In [ ]:

print procLine[:5]
print clusterDict['1']


In [ ]:
import itertools

for p in procLine:
    l = p.split(' ')
    if len(l) > 1:
        comb = itertools.combinations(l, 2)
        for start,finish in comb:
            val = (start,finish)
            edgeDict[val] += 1
            edgeSet.add(val)
    

In [ ]:
import math
dataOutFile = '/Users/dgrossman/work/magichour/d3/data3.csv'
header = 'source,target,value,sTitle,tTitle\n'
outFile = open(dataOutFile,'w')
outFile.write(header)
for edge,count in edgeDict.iteritems():
    o =  '%s,%s,%f,%s %s,%s %s\n' % (edge[0],edge[1],math.log(int(count))+1,
                                     edge[0],clusterDict[edge[0]],
                                     edge[1],clusterDict[edge[1]])
    outFile.write(o)
outFile.close()

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def draw_graph(graph):

    # extract nodes from graph
    nodes = set([n1 for n1, n2 in graph] + [n2 for n1, n2 in graph])

    # create networkx graph
    G=nx.Graph()

    # add nodes
    for node in nodes:
        G.add_node(node)

    # add edges
    for edge in graph:
        G.add_edge(edge[0], edge[1])

    # draw graph
    pos = nx.spring_layout(G,k=2)
    nx.draw(G, pos)

    # show graph
    plt.show()

# draw example
graph = edges
draw_graph(graph)

In [ ]:
import networkx as nx
from wordcloud import WordCloud
import matplotlib.pyplot as plt


G=nx.Graph()
edges = list()
for x in edgeDict.iterkeys():
    edges.append(x)
G.add_edges_from(edges)

nodeweight = defaultdict(int)

for k,v in edgeDict.iteritems():
    nodeweight[k[0]] += v
    nodeweight[k[1]] += v




components = nx.connected_components(G)
temp =0
for x in components:
    temp = temp+1

numFigs = math.ceil(math.sqrt(temp))

fig = plt.figure()

components = nx.connected_components(G)

temp = 1
for c in components:
    wordWeight = defaultdict(int)
    wordList = list()
    for cID in c:    
        for word in clusterDict[cID].split():
            if word not in  [' ','{','}','[',']',':'] and word.islower():
                wordWeight[word] += nodeweight[cID]
    for k,v in wordWeight.iteritems():
        wordList.append((k,v))
    #print len(wordList)
    #print wordList
    print 'image'
    print wordList[:10]
    wordcloud = WordCloud(max_font_size=40, relative_scaling=.5).generate_from_frequencies(wordList)
        # print wordWeight
    plt.subplot(numFigs,numFigs,temp)
    plt.title('component %i' % temp)
    plt.imshow(wordcloud)
    plt.axis("off")
    temp=temp+1
plt.show()

